# 타이타닉 데이터넷으로 전처리 연습

In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
train = pd.read_csv('https://bit.ly/fc-ml-titanic')

In [7]:
# csv 파일에서 불러온 데이터셋의 가장 앞의 5개(default) 데이터를 보여준다
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


* Passengerid : 승객 탑승 아이디
* Survived : 생존여부, 1생존, 0사망
* Pclass : 선실의 등급
* Name : 이름
* Sex : 성별
* Age : 나이
* SibSp : 형제, 자매, 배우자 수 (수평관계)
* Parch : 부모, 자식 수 (수직관계)
* Ticket : 티켓 번호
* Fare : 요금
* Cabin : 좌석번호
* Embarked : 탑승 항구

### 전처리 : train / vaildation 데이터넷 나누기
1. feature와 label을 정의
2. feature / label을 적절한 비율로 나눈다.

In [10]:
# 전체 데이터중 학습에 사용할 데이터를 선택
feature = ['Pclass', 'Sex', 'Age', 'Fare']

# 정답값(train) 혹은 예측값과 비교할 값(test)
label = ['Survived']

In [11]:
# feature를 정의하고 train[feature]를 지정해주면 리스트안에 선택된 항목만 가져옴.
train[feature].head()

,Pclass,Sex,Age,Fare
0,3,male,22.0,7.2500
1,1,female,38.0,71.2833
2,3,female,26.0,7.9250
3,1,female,35.0,53.1000
4,3,male,35.0,8.0500


In [12]:
train[label].head()

,Survived
0,0
1,1
2,1
3,1
4,0


In [13]:
# train의 데이터 특성을 확인해보자
train.info()

# 결측치를 해결해야 겠는걸

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [23]:
x_train, x_valid, y_train, y_valid = train_test_split(train[feature], train[label], test_size=0.2, random_state=123)

In [24]:
print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(712, 4)
(712, 1)
(179, 4)
(179, 1)


### 전처리 : 결측치를 처리

In [25]:
# pandas의 insull() 를 사용하여
# 결측치의 전체 갯수 확인
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [27]:
train['Age'].isnull().sum()

177

### 수치형 결측치 데이터에 대한 처리

수치형 데이터의 결측치는 대체로 평균값을 구한 뒤, 평균값으로 채워준다.

In [28]:
# 0으로 채웠을때
# describ() 통계치 데이터를 보여준다
train['Age'].fillna(0).describe()

count    891.000000
mean      23.799293
std       17.596074
min        0.000000
25%        6.000000
50%       24.000000
75%       35.000000
max       80.000000
Name: Age, dtype: float64

In [30]:
# 평균값으로 채웠을때
meanAge = train['Age'].mean()
print(meanAge)
train['Age'].fillna(meanAge).describe()

29.69911764705882


count    891.000000
mean      29.699118
std       13.002015
min        0.420000
25%       22.000000
50%       29.699118
75%       35.000000
max       80.000000
Name: Age, dtype: float64

### scikit-learn 의 imputer 메서드
를 활용하면 2개 이상의 컬럼을 한 번에 처리할 수 있다.

In [39]:
from sklearn.impute import SimpleImputer

# imputer 모델선언
imputer = SimpleImputer(strategy='mean')

In [40]:
# fit()
imputer.fit(train[['Age', 'Pclass']])

SimpleImputer()

In [41]:
# transform()
result = imputer.transform(train[['Age', 'Pclass']])

In [42]:
result

array([[22.        ,  3.        ],
       [38.        ,  1.        ],
       [26.        ,  3.        ],
       ...,
       [29.69911765,  3.        ],
       [26.        ,  1.        ],
       [32.        ,  3.        ]])

In [43]:
# update
train[['Age', 'Pclass']] = result

In [45]:
train[['Age', 'Pclass']].isnull().sum()

Age       0
Pclass    0
dtype: int64

In [47]:
train[['Age', 'Pclass']].describe()

,Age,Pclass
count,891.000000,891.000000
mean,29.699118,2.308642
std,13.002015,0.836071
min,0.420000,1.000000
25%,22.000000,2.000000
50%,29.699118,3.000000
75%,35.000000,3.000000
max,80.000000,3.000000


In [49]:
# 결측치가 있던 원본데이터로 복귀
train = pd.read_csv('https://bit.ly/fc-ml-titanic')

In [51]:
train[['Age', 'Pclass']].isnull().sum()

Age       177
Pclass      0
dtype: int64

In [54]:
# imputer model 정의
imputer = SimpleImputer(strategy='mean')

In [62]:
# fit() + transform() => #fit_transform()
result = imputer.fit_transform(train[['Age', 'Pclass']])

In [63]:
train[['Age', 'Pclass']] = result

In [64]:
train[['Age', 'Pclass']].isnull().sum()

Age       0
Pclass    0
dtype: int64

In [65]:
train[['Age', 'Pclass']].describe()

,Age,Pclass
count,891.000000,891.000000
mean,29.699118,2.308642
std,13.002015,0.836071
min,0.420000,1.000000
25%,22.000000,2.000000
50%,29.699118,3.000000
75%,35.000000,3.000000
max,80.000000,3.000000


### Categorical 데이터에 대한 결측치 처리

In [71]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [74]:
# Embarked
# train['Embarked'].fillna('S')

imputer = SimpleImputer(strategy='most_frequent')
result = imputer.fit_transform(train[['Embarked', 'Cabin']])
train[['Embarked', 'Cabin']] = result

In [77]:
train[['Embarked', 'Cabin']].isnull().sum()

Embarked    0
Cabin       0
dtype: int64

### Label Encoding : 문자를 수치로 변환
학습을 할 때는 문자로된 데이터를 수치로 변환해야 합니다.

In [78]:
def convert(data):
    if data=='male':
        return 1
    elif data=='female':
        return 0

In [79]:
train['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [80]:
train['Sex'].apply(convert)

0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Name: Sex, Length: 891, dtype: int64

In [81]:
from sklearn.preprocessing import LabelEncoder

# LabelEncoder 모델정의
le = LabelEncoder()

In [87]:
# fit_transfrom()
train['Sex_num'] = le.fit_transform(train['Sex'])

In [89]:
le.classes_

array(['female', 'male'], dtype=object)

In [90]:
le.inverse_transform([0,1,1,0])

array(['female', 'male', 'male', 'female'], dtype=object)

In [86]:
train['Sex_num'].value_counts()

1    577
0    314
Name: Sex_num, dtype: int64

In [98]:
# 'S'. 'C', 'Q' class
train['Embaked_num'] = le.fit_transform(train['Embarked'])

In [110]:
train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [94]:
le.classes_

array(['C', 'Q', 'S'], dtype=object)

### One-hot Encoding

In [100]:
# 결측치가 있던 원본데이터로 복귀
train = pd.read_csv('https://bit.ly/fc-ml-titanic')

In [102]:
train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [104]:
train['Embarked'][:6]

0    S
1    C
2    S
3    S
4    S
5    Q
Name: Embarked, dtype: object

In [118]:
# 문자열 -> labelEncoder()변환 -> one-hot Encoding

train['Embarked_num'] = LabelEncoder().fit_transform(train['Embarked'])

In [119]:
train['Embarked_num'].value_counts()

2    644
0    168
1     77
3      2
Name: Embarked_num, dtype: int64

In [115]:
# pandas.get_dummies()
pd.get_dummies(train['Embarked'][:6])

,C,Q,S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
5,0,1,0


In [121]:
one_hot = pd.get_dummies(train['Embarked_num'])

In [123]:
one_hot.columns = ['C', 'Q', 'S', 'N']

In [125]:
one_hot

,C,Q,S,N
0,0,0,1,0
1,1,0,0,0
2,0,0,1,0
3,0,0,1,0
4,0,0,1,0
...,...,...,...,...
886,0,0,1,0
887,0,0,1,0
888,0,0,1,0
889,1,0,0,0


### 전처리 : Normalize(정규화)

영화평점 리뷰
네이버 평점 리뷰 (0점~10점) : (2, 4, 6, 8, 10)
넷플릭스 평점 리뷰 (0점~5점) : (1, 2, 3, 4, 5 )

데이터간에 다른 min, max 값을 가지는 경우, 정규화를 통해 최소/최대치 값의 척도를 맞추어준다.

In [129]:
import pandas as pd

movie = {'naver' : [2, 4, 6, 8, 10],
         'netflix' : [1, 2, 3, 4, 5]
         }

In [135]:
movies = pd.DataFrame(data=movie)
movies

,naver,netflix
0,2,1
1,4,2
2,6,3
3,8,4
4,10,5


In [138]:
from sklearn.preprocessing import MinMaxScaler

In [141]:
# 모델선언
min_max_scaler = MinMaxScaler()

In [143]:
min_max_movie = min_max_scaler.fit_transform(movies)

In [146]:
# min=0, max=1
pd.DataFrame(min_max_movie, columns=['naver', 'netflix'])

,naver,netflix
0,0.00,0.00
1,0.25,0.25
2,0.50,0.50
3,0.75,0.75
4,1.00,1.00
